In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
#btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
#btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

In [3]:
vol0 = get_single_volume(btf_fname_red, 0, num_slices=33, alpha=0.15)

In [4]:
alpha = 0.15
dat_foldername = r'..\point_cloud_alignment'
fname = os.path.join(dat_foldername, 'img100.tif')
f = lambda tif : (alpha*tif.asarray()).astype('uint8')
with tifffile.TiffFile(fname) as tif0:
    vol0_old = f(tif0)

## Use single function to track

In [5]:
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *


In [9]:
all_matches, all_conf, all_neurons = track_neurons_full_video(btf_fname_red,
                             start_frame=0,
                             num_frames=500,
                             num_slices=33,
                             alpha=0.15,
                             use_mini_max_projection=True,
                             verbose=2)

Matching frames 0 and 1 (end at 500)
Matching frames 1 and 2 (end at 500)
Matching frames 2 and 3 (end at 500)
Matching frames 3 and 4 (end at 500)
Matching frames 4 and 5 (end at 500)
Matching frames 5 and 6 (end at 500)
Matching frames 6 and 7 (end at 500)
Matching frames 7 and 8 (end at 500)
Matching frames 8 and 9 (end at 500)
Matching frames 9 and 10 (end at 500)
Matching frames 10 and 11 (end at 500)
Matching frames 11 and 12 (end at 500)
Matching frames 12 and 13 (end at 500)
Matching frames 13 and 14 (end at 500)
Matching frames 14 and 15 (end at 500)
Matching frames 15 and 16 (end at 500)
Matching frames 16 and 17 (end at 500)
Matching frames 17 and 18 (end at 500)
Matching frames 18 and 19 (end at 500)
Matching frames 19 and 20 (end at 500)
Matching frames 20 and 21 (end at 500)
Matching frames 21 and 22 (end at 500)
Matching frames 22 and 23 (end at 500)
Matching frames 23 and 24 (end at 500)
Matching frames 24 and 25 (end at 500)
Matching frames 25 and 26 (end at 500)
Match

Matching frames 83 and 84 (end at 500)
Matching frames 84 and 85 (end at 500)
Matching frames 85 and 86 (end at 500)
Matching frames 86 and 87 (end at 500)
Matching frames 87 and 88 (end at 500)
Matching frames 88 and 89 (end at 500)
Matching frames 89 and 90 (end at 500)
Matching frames 90 and 91 (end at 500)
Matching frames 91 and 92 (end at 500)
Matching frames 92 and 93 (end at 500)
Matching frames 93 and 94 (end at 500)
Matching frames 94 and 95 (end at 500)
Matching frames 95 and 96 (end at 500)
Matching frames 96 and 97 (end at 500)
Matching frames 97 and 98 (end at 500)
Matching frames 98 and 99 (end at 500)
Matching frames 99 and 100 (end at 500)
Matching frames 100 and 101 (end at 500)
Matching frames 101 and 102 (end at 500)
Matching frames 102 and 103 (end at 500)
Matching frames 103 and 104 (end at 500)
Matching frames 104 and 105 (end at 500)
Matching frames 105 and 106 (end at 500)
Matching frames 106 and 107 (end at 500)
Matching frames 107 and 108 (end at 500)
Matching

Matching frames 165 and 166 (end at 500)
Matching frames 166 and 167 (end at 500)
Matching frames 167 and 168 (end at 500)
Matching frames 168 and 169 (end at 500)
Matching frames 169 and 170 (end at 500)
Matching frames 170 and 171 (end at 500)
Matching frames 171 and 172 (end at 500)
Matching frames 172 and 173 (end at 500)
Matching frames 173 and 174 (end at 500)
Matching frames 174 and 175 (end at 500)
Matching frames 175 and 176 (end at 500)
Matching frames 176 and 177 (end at 500)
Matching frames 177 and 178 (end at 500)
Matching frames 178 and 179 (end at 500)
Matching frames 179 and 180 (end at 500)
Matching frames 180 and 181 (end at 500)
Matching frames 181 and 182 (end at 500)
Matching frames 182 and 183 (end at 500)
Matching frames 183 and 184 (end at 500)
Matching frames 184 and 185 (end at 500)
Matching frames 185 and 186 (end at 500)
Matching frames 186 and 187 (end at 500)
Matching frames 187 and 188 (end at 500)
Matching frames 188 and 189 (end at 500)
Matching frames 

Matching frames 247 and 248 (end at 500)
Matching frames 248 and 249 (end at 500)
Matching frames 249 and 250 (end at 500)
Matching frames 250 and 251 (end at 500)
Matching frames 251 and 252 (end at 500)
Matching frames 252 and 253 (end at 500)
Matching frames 253 and 254 (end at 500)
Matching frames 254 and 255 (end at 500)
Matching frames 255 and 256 (end at 500)
Matching frames 256 and 257 (end at 500)
Matching frames 257 and 258 (end at 500)
Matching frames 258 and 259 (end at 500)
Matching frames 259 and 260 (end at 500)
Matching frames 260 and 261 (end at 500)
Matching frames 261 and 262 (end at 500)
Matching frames 262 and 263 (end at 500)
Matching frames 263 and 264 (end at 500)
Matching frames 264 and 265 (end at 500)
Matching frames 265 and 266 (end at 500)
Matching frames 266 and 267 (end at 500)
Matching frames 267 and 268 (end at 500)
Matching frames 268 and 269 (end at 500)
Matching frames 269 and 270 (end at 500)
Matching frames 270 and 271 (end at 500)
Matching frames 

Matching frames 329 and 330 (end at 500)
Matching frames 330 and 331 (end at 500)
Matching frames 331 and 332 (end at 500)
Matching frames 332 and 333 (end at 500)
Matching frames 333 and 334 (end at 500)
Matching frames 334 and 335 (end at 500)
Matching frames 335 and 336 (end at 500)
Matching frames 336 and 337 (end at 500)
Matching frames 337 and 338 (end at 500)
Matching frames 338 and 339 (end at 500)
Matching frames 339 and 340 (end at 500)
Matching frames 340 and 341 (end at 500)
Matching frames 341 and 342 (end at 500)
Matching frames 342 and 343 (end at 500)
Matching frames 343 and 344 (end at 500)
Matching frames 344 and 345 (end at 500)
Matching frames 345 and 346 (end at 500)
Matching frames 346 and 347 (end at 500)
Matching frames 347 and 348 (end at 500)
Matching frames 348 and 349 (end at 500)
Matching frames 349 and 350 (end at 500)
Matching frames 350 and 351 (end at 500)
Matching frames 351 and 352 (end at 500)
Matching frames 352 and 353 (end at 500)
Matching frames 

Matching frames 411 and 412 (end at 500)
Matching frames 412 and 413 (end at 500)
Matching frames 413 and 414 (end at 500)
Matching frames 414 and 415 (end at 500)
Matching frames 415 and 416 (end at 500)
Matching frames 416 and 417 (end at 500)
Matching frames 417 and 418 (end at 500)
Matching frames 418 and 419 (end at 500)
Matching frames 419 and 420 (end at 500)
Matching frames 420 and 421 (end at 500)
Matching frames 421 and 422 (end at 500)
Matching frames 422 and 423 (end at 500)
Matching frames 423 and 424 (end at 500)
Matching frames 424 and 425 (end at 500)
Matching frames 425 and 426 (end at 500)
Matching frames 426 and 427 (end at 500)
Matching frames 427 and 428 (end at 500)
Matching frames 428 and 429 (end at 500)
Matching frames 429 and 430 (end at 500)
Matching frames 430 and 431 (end at 500)
Matching frames 431 and 432 (end at 500)
Matching frames 432 and 433 (end at 500)
Matching frames 433 and 434 (end at 500)
Matching frames 434 and 435 (end at 500)
Matching frames 

Matching frames 493 and 494 (end at 500)
Matching frames 494 and 495 (end at 500)
Matching frames 495 and 496 (end at 500)
Matching frames 496 and 497 (end at 500)
Matching frames 497 and 498 (end at 500)
Matching frames 498 and 499 (end at 500)
Finished 500 frames in 7307.301607847214 seconds


In [10]:
clust_df = build_tracklets_from_matches(all_neurons, all_matches, verbose=1)

0 / 499
1 / 499
Finished tracks [  0   5   6  16  20  30  32  34  53  56  60  65  68  75  78  79  82  94
  97 100]
2 / 499
Finished tracks [  0   1   2   4   5   6   7   8   9  10  11  12  14  16  18  19  20  21
  22  23  24  25  26  28  29  30  31  32  33  34  35  36  37  39  40  42
  43  44  46  47  48  49  51  52  53  55  56  58  60  61  62  63  64  65
  67  68  69  70  71  72  75  76  77  78  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  93  94  96  97  98  99 100 102 103 105 106 107
 108 109 111 112 113 114 115 117 119 120 121 123 125 126 127 129 130 131
 132 133 134 135 136 137 138 139 141 144 145]
3 / 499
Finished tracks [  0   1   2   4   5   6   7   8   9  10  11  12  13  14  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  39  40  42  43  44  46  47  48  49  50  51  52  53  55  56  58  60  61
  62  63  64  65  67  68  69  70  71  72  75  76  77  78  79  80  81  82
  83  84  85  86  87  88  89  90  91  92  93  94  96  97  98 

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 234 235 236 237
 238 240 241 242 243 244 245 246 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 234
 235 236 237 238 239 240 241 242 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 234
 235 236 237 238 239 240 241 242 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 234
 235 236 237 238 239 240 241 242 24

...

Finished tracks [   0    1    2 ... 2892 2898 2900]
192 / 499
Finished tracks [   0    1    2 ... 2900 2903 2905]
193 / 499
Finished tracks [   0    1    2 ... 2908 2909 2910]
194 / 499
Finished tracks [   0    1    2 ... 2910 2917 2920]
195 / 499
Finished tracks [   0    1    2 ... 2927 2929 2939]
196 / 499
Finished tracks [   0    1    2 ... 2947 2948 2949]
197 / 499
Finished tracks [   0    1    2 ... 2957 2961 2964]
198 / 499
Finished tracks [   0    1    2 ... 2961 2964 2975]
199 / 499
Finished tracks [   0    1    2 ... 2998 2999 3006]
200 / 499
Finished tracks [   0    1    2 ... 3006 3009 3010]
201 / 499
Finished tracks [   0    1    2 ... 3027 3028 3029]
202 / 499
Finished tracks [   0    1    2 ... 3033 3036 3040]
203 / 499
Finished tracks [   0    1    2 ... 3046 3048 3049]
204 / 499
Finished tracks [   0    1    2 ... 3048 3049 3054]
205 / 499
Finished tracks [   0    1    2 ... 3074 3077 3079]
206 / 499
Finished tracks [   0    1    2 ... 3077 3079 3093]
207 / 499
Finished

Finished tracks [   0    1    2 ... 3677 3679 3680]
258 / 499
Finished tracks [   0    1    2 ... 3685 3686 3687]
259 / 499
Finished tracks [   0    1    2 ... 3685 3686 3687]
260 / 499
Finished tracks [   0    1    2 ... 3688 3689 3696]
261 / 499
Finished tracks [   0    1    2 ... 3696 3705 3709]
262 / 499
Finished tracks [   0    1    2 ... 3699 3705 3709]
263 / 499
Finished tracks [   0    1    2 ... 3706 3709 3722]
264 / 499
Finished tracks [   0    1    2 ... 3715 3722 3723]
265 / 499
Finished tracks [   0    1    2 ... 3729 3730 3731]
266 / 499
Finished tracks [   0    1    2 ... 3729 3730 3731]
267 / 499
Finished tracks [   0    1    2 ... 3730 3731 3737]
268 / 499
Finished tracks [   0    1    2 ... 3746 3753 3764]
269 / 499
Finished tracks [   0    1    2 ... 3764 3766 3767]
270 / 499
Finished tracks [   0    1    2 ... 3778 3779 3780]
271 / 499
Finished tracks [   0    1    2 ... 3792 3793 3795]
272 / 499
Finished tracks [   0    1    2 ... 3795 3797 3803]
273 / 499
Finished

Finished tracks [   0    1    2 ... 4408 4409 4410]
324 / 499
Finished tracks [   0    1    2 ... 4408 4409 4410]
325 / 499
Finished tracks [   0    1    2 ... 4409 4410 4415]
326 / 499
Finished tracks [   0    1    2 ... 4415 4423 4429]
327 / 499
Finished tracks [   0    1    2 ... 4423 4428 4429]
328 / 499
Finished tracks [   0    1    2 ... 4439 4441 4443]
329 / 499
Finished tracks [   0    1    2 ... 4443 4450 4452]
330 / 499
Finished tracks [   0    1    2 ... 4450 4452 4455]
331 / 499
Finished tracks [   0    1    2 ... 4450 4452 4455]
332 / 499
Finished tracks [   0    1    2 ... 4461 4464 4473]
333 / 499
Finished tracks [   0    1    2 ... 4476 4478 4483]
334 / 499
Finished tracks [   0    1    2 ... 4483 4484 4486]
335 / 499
Finished tracks [   0    1    2 ... 4500 4502 4503]
336 / 499
Finished tracks [   0    1    2 ... 4519 4520 4521]
337 / 499
Finished tracks [   0    1    2 ... 4530 4535 4538]
338 / 499
Finished tracks [   0    1    2 ... 4538 4547 4553]
339 / 499
Finished

Finished tracks [   0    1    2 ... 5072 5073 5075]
390 / 499
Finished tracks [   0    1    2 ... 5075 5079 5080]
391 / 499
Finished tracks [   0    1    2 ... 5079 5080 5102]
392 / 499
Finished tracks [   0    1    2 ... 5093 5101 5102]
393 / 499
Finished tracks [   0    1    2 ... 5102 5107 5118]
394 / 499
Finished tracks [   0    1    2 ... 5118 5124 5131]
395 / 499
Finished tracks [   0    1    2 ... 5131 5137 5139]
396 / 499
Finished tracks [   0    1    2 ... 5137 5139 5141]
397 / 499
Finished tracks [   0    1    2 ... 5142 5144 5150]
398 / 499
Finished tracks [   0    1    2 ... 5142 5144 5150]
399 / 499
Finished tracks [   0    1    2 ... 5158 5162 5165]
400 / 499
Finished tracks [   0    1    2 ... 5158 5162 5165]
401 / 499
Finished tracks [   0    1    2 ... 5165 5166 5167]
402 / 499
Finished tracks [   0    1    2 ... 5169 5171 5183]
403 / 499
Finished tracks [   0    1    2 ... 5180 5181 5183]
404 / 499
Finished tracks [   0    1    2 ... 5183 5188 5191]
405 / 499
Finished

Finished tracks [   0    1    2 ... 5619 5630 5638]
456 / 499
Finished tracks [   0    1    2 ... 5630 5638 5648]
457 / 499
Finished tracks [   0    1    2 ... 5652 5659 5662]
458 / 499
Finished tracks [   0    1    2 ... 5659 5660 5662]
459 / 499
Finished tracks [   0    1    2 ... 5663 5666 5673]
460 / 499
Finished tracks [   0    1    2 ... 5675 5676 5684]
461 / 499
Finished tracks [   0    1    2 ... 5679 5683 5684]
462 / 499
Finished tracks [   0    1    2 ... 5684 5695 5697]
463 / 499
Finished tracks [   0    1    2 ... 5703 5704 5705]
464 / 499
Finished tracks [   0    1    2 ... 5703 5704 5705]
465 / 499
Finished tracks [   0    1    2 ... 5719 5727 5745]
466 / 499
Finished tracks [   0    1    2 ... 5748 5752 5755]
467 / 499
Finished tracks [   0    1    2 ... 5749 5752 5755]
468 / 499
Finished tracks [   0    1    2 ... 5752 5755 5767]
469 / 499
Finished tracks [   0    1    2 ... 5775 5777 5778]
470 / 499
Finished tracks [   0    1    2 ... 5776 5777 5778]
471 / 499
Finished

## Save

In [11]:
import pickle

In [12]:
fname = 'clust_df_dat_minimax.pickle'
pickle.dump(clust_df,open(fname,'wb'))

In [13]:
fname = 'match_dat_minimax.pickle'
pickle.dump((all_matches, all_conf, all_neurons), open(fname,'wb'))

## Visualize

In [14]:
i=0

neurons0 = all_neurons[i]
neurons1 = all_neurons[i+1]
matches = all_matches[i]

visualize_tracks(neurons0, neurons1, matches)

In [15]:
all_conf[0]

[0.4230769230769231,
 0.6451612903225806,
 0.3333333333333333,
 0.3728813559322034,
 0.1111111111111111,
 0.3125,
 0.2,
 0.4444444444444444,
 0.5757575757575758,
 0.2857142857142857,
 0.21428571428571427,
 0.2857142857142857,
 0.7428571428571429,
 0.25,
 0.6779661016949152,
 0.21739130434782608,
 0.3050847457627119,
 0.5084745762711864,
 0.423728813559322,
 0.17857142857142858,
 0.3142857142857143,
 0.5471698113207547,
 0.4745762711864407,
 0.3333333333333333,
 0.23529411764705882,
 0.16,
 0.8305084745762712,
 0.543859649122807,
 0.5660377358490566,
 0.46875,
 0.41379310344827586,
 0.32142857142857145,
 0.4,
 0.6206896551724138,
 0.23333333333333334,
 0.2,
 0.603448275862069,
 0.3333333333333333,
 0.5084745762711864,
 0.6949152542372882,
 0.4406779661016949,
 0.3389830508474576,
 0.47058823529411764,
 0.4727272727272727,
 0.5932203389830508,
 0.4915254237288136,
 0.5217391304347826,
 0.7627118644067796,
 0.5909090909090909,
 0.4576271186440678,
 0.18181818181818182,
 0.2222222222222222

# Aside: timing tests

In [15]:
# Open video
def open_tif(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        1+1

def open_page(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat = page.asarray()
            if i>0:
                break

def open_volume(vid_fname, num_slices=33):
    dat = []
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat.append(page.asarray())
            if i>num_slices:
                break
                
def open_with_imread(vid_fname, num_slices=33):
    dat = tifffile.imread(vid_fname, key=range(num_slices))

In [9]:
%timeit open_tif(btf_fname_red)

269 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit open_page(btf_fname_red)

2 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit open_volume(btf_fname_red)

38.8 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit open_with_imread(btf_fname_red)

31.5 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
